In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import autokeras as ak
import tensorflow as tf
import os
from tensorboard.plugins.hparams import api as hp
%load_ext tensorboard

In [2]:
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D , Lambda, Conv2D, Activation,Concatenate
from tensorflow.keras.layers import ActivityRegularization, BatchNormalization
from tensorflow.keras.optimizers import Adam , SGD , Adagrad
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers , initializers
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import NumpyArrayIterator
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import backend as K

In [3]:
import logging

logging.basicConfig(level = logging.INFO)
gpus = tf.config.experimental.list_physical_devices('GPU')
# if gpus:
#   # Restrict TensorFlow to only use the first GPU
try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    tf.config.experimental.set_virtual_device_configuration(
    gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5000)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
#     logging.info(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
except RuntimeError as e:
# Visible devices must be set before GPUs have been initialized
    logging.info(e)
    
    
logging.info("Tensorflow Version is {}".format(tf.__version__))
logging.info("Keras Version is {}".format(tf.keras.__version__))
from tensorflow.python.client import device_lib
logging.info(device_lib.list_local_devices())
tf.device('/device:XLA_GPU:0')

INFO:root:Tensorflow Version is 2.4.0-rc3
INFO:root:Keras Version is 2.4.0
INFO:root:[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1218153156013619229
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5242880000
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8366061891710613685
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 0000:03:00.0, compute capability: 7.5"
]


In [4]:
data = np.load('/home/ML4NO/Data/n1000000_0910_classification.npz')

In [5]:
data_all = np.column_stack([data['ve_dune'][:,36:], data['vu_dune'][:,36:], data['vebar_dune'][:,36:], data['vubar_dune'][:,36:]])
target = data['cpv']

x_train = data_all[:10000]
y_train = target[:10000]

x_train2 = data_all[10000:900000]
y_train2 = target[10000:900000]

x_test = data_all[900000:]
y_test = target[900000:]

In [6]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([128,256,512,1024]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([128,256,512,1024]))
HP_NUM_UNITS3 = hp.HParam('num_units3', hp.Discrete([128,256,512,1024]))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('test/highE/hparam_tuning').as_default():
    hp.hparams_config(
    hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_NUM_UNITS3],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [7]:
def train_test_model(hparams):
    model_DNN = Sequential(name = "Model_DNN")  
    model_DNN.add(keras.Input(shape=(len(x_train[0]),), name = 'input'))
    model_DNN.add(BatchNormalization(name = 'BatchNormalization'))
    model_DNN.add(Dense(hparams[HP_NUM_UNITS1], activation='relu', name = 'dense_1'))
    model_DNN.add(Dense(hparams[HP_NUM_UNITS2], activation='relu', name = 'dense_2'))
    model_DNN.add(Dense(hparams[HP_NUM_UNITS3], activation='relu', name = 'dense_3'))
    model_DNN.add(Dense(1, activation='sigmoid', name = 'output'))
    model_opt = keras.optimizers.Adam()
    model_DNN.compile(loss="binary_crossentropy",
                              optimizer=model_opt,
                              metrics=['accuracy'])
    model_DNN.fit(x_train2, y_train2,
           validation_split = 0.1,
           batch_size=64,
           epochs=5,
           verbose=1,
           shuffle = True
         )
    _, accuracy = model_DNN.evaluate(x_test, y_test)
    return accuracy

In [8]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [9]:
session_num = 0

for num_units1 in HP_NUM_UNITS1.domain.values:
    for num_units2 in HP_NUM_UNITS2.domain.values:
        for num_units3 in HP_NUM_UNITS3.domain.values:
            hparams = {
            HP_NUM_UNITS1: num_units1,
            HP_NUM_UNITS2: num_units2,
            HP_NUM_UNITS3: num_units3,
        }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('test/highE/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'num_units1': 128, 'num_units2': 128, 'num_units3': 128}
Epoch 1/5
12516/12516 [==============================] - 72s 6ms/step - loss: 0.4702 - accuracy: 0.7665 - val_loss: 0.3047 - val_accuracy: 0.8736
Epoch 2/5
12516/12516 [==============================] - 69s 6ms/step - loss: 0.3905 - accuracy: 0.8247 - val_loss: 0.3350 - val_accuracy: 0.8582
Epoch 3/5
12516/12516 [==============================] - 70s 6ms/step - loss: 0.3803 - accuracy: 0.8303 - val_loss: 0.2931 - val_accuracy: 0.8758
Epoch 4/5
12516/12516 [==============================] - 70s 6ms/step - loss: 0.3722 - accuracy: 0.8355 - val_loss: 0.3010 - val_accuracy: 0.8695
Epoch 5/5
3125/3125 [==============================] - 10s 3ms/step - loss: 0.3087 - accuracy: 0.8680
--- Starting trial: run-1
{'num_units1': 128, 'num_units2': 128, 'num_units3': 256}
Epoch 1/5
12516/12516 [==============================] - 73s 6ms/step - loss: 0.4663 - accuracy: 0.7677 - val_loss: 0.3372 - val_accuracy: 0.8539


In [10]:
%tensorboard --logdir 'test/highE' --bind_all